# Driving, walking, biking skims

Compute driving time from every tract to every other tract in Los Angeles, using OSRM. Note that `--max-table-size=5000` or so is required when starting osrm-routed. This should be run first, followed by `Congestion model` and `Transit model`.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import geopandas as gp
import openmatrix as omx

DB_URI = 'postgresql://matthewc@localhost/matthewc'
OSRM_DRIVE_URI = 'http://localhost:5000'
OSRM_WALK_URI = 'http://localhost:5001'

In [ ]:
# ORDER BY to force stable order 
tract_centroids = gp.read_postgis('''
    SELECT geoid, ST_Centroid(geog)::geometry AS geom
    FROM diss.ca_tracts
    WHERE countyfp in ('025', '037', '059', '065', '071', '111')
    ORDER BY geoid''', DB_URI)

## Initialize OpenMatrix file

In [ ]:
skims = omx.open_file('../la_abm/data/skims.omx', 'w')  # overwrite

In [ ]:
tract_centroids['idx'] = np.arange(len(tract_centroids))

In [ ]:
tract_centroids[['geoid', 'idx']].to_parquet('../la_abm/data/skim_tracts.parquet')

## Retrieve distance and free-flow travel time for driving

In [ ]:
request_url = f'{OSRM_DRIVE_URI}/table/v1/driving/' + ';'.join(map(lambda x: '{},{}'.format(*x), zip(tract_centroids.geom.x, tract_centroids.geom.y))) + '?annotations=duration,distance'
print(request_url[:500])

In [ ]:
%time raw_dmat = requests.get(request_url).json()

In [ ]:
# create base dmat and convert to minutes
# TODO what about distances?
base_dmat = np.array(raw_dmat['durations']) / 60

In [ ]:
skims['car_freeflow'] = base_dmat

In [ ]:
skims['car_distance_km'] = np.array(raw_dmat['distances']) / 1000

In [ ]:
np.array(skims['car_distance_km'])

## Retrieve distance and free-flow times for walking

In [ ]:
request_url = f'{OSRM_WALK_URI}/table/v1/walking/' + ';'.join(map(lambda x: '{},{}'.format(*x), zip(tract_centroids.geom.x, tract_centroids.geom.y))) + '?annotations=duration,distance'
%time raw_dmat = requests.get(request_url).json()
# astype because there are some Nones here, make them nans
skims['walk'] = np.array(raw_dmat['durations']).astype('float64') / 60
skims['walk_dist_km'] = np.array(raw_dmat['distances']).astype('float64') / 1000

In [ ]:
skims.close()